# initial tests

In [3]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [30]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

API_KEY=os.getenv("API_KEY")
MODEL="gpt-3.5-turbo-0125"

def listModels():
    url = "https://api.openai.com/v1/models"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + API_KEY,
    }
    response = requests.get(url, headers=headers)
    print(response.json())
def chatGPT(text):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + API_KEY,
    }
    data = {
        "model": MODEL,
        "messages": [
          {
            "role": "system",
            "content": "You are a tool used to automatically generate source code for unit tests. Source code for a function that is to be tested will be provided and you should generate an appropriate unit tests that can be ran to ensure the code is correct. Code input will be in Java and you should respond with a Java code block containing the unit test as output. Use org.junit.jupiter as the testing framework. Aim for maximum coverage. Don't assume Equals methods have been correctly overidden. Don't make use of immutable Collections as inputs unless absolutely necessary."
          },
          {
            "role": "user",
            "content": """
              public class TestClass {
                public int add(int a, int b) {
                    return a+b;
                }
                public int subtract(int a, int b) {
                    return a-b;
                }
            }
              """
          },
          {
            "role": "assistant",
            "content": "package com.unit.test;\n\nimport org.junit.Test;\nimport static org.junit.Assert.assertEquals;\n\npublic class TestClassTest {\n\n    @Test\n    public void testAdd() {\n        TestClass testClass = new TestClass();\n        int result = testClass.add(5, 3);\n        assertEquals(8, result);\n    }\n\n    @Test\n    public void testSubtract() {\n        TestClass testClass = new TestClass();\n        int result = testClass.subtract(10, 5);\n        assertEquals(5, result);\n    }\n}"
          },
          {
            "role": "user",
            "content": text
          },
        ],
        "max_tokens": 4096,
        "temperature": 1.0,
    }
    response = requests.post(url, headers=headers, json=data)
    print(response.json())
    output = response.json()['choices'][0]['message']['content']

    return output


test = chatGPT("""
        package emu.grasscutter.command;

        import emu.grasscutter.game.world.Position;
        import java.util.*;
        import java.util.function.BiConsumer;
        import java.util.regex.*;
        import javax.annotation.Nonnull;

        public class CommandHelpers {
            public static final Pattern lvlRegex =
                    Pattern.compile("(?<!\\w)l(?:vl?)?(\\d+)"); // Java doesn't have raw string literals :(
            public static final Pattern amountRegex =
                    Pattern.compile("((?<=(?<!\\w)x)\\d+|\\d+(?=x)(?!x\\d))");
            public static final Pattern refineRegex = Pattern.compile("(?<!\\w)r(\\d+)");
            public static final Pattern rankRegex = Pattern.compile("(\\d+)\\*");
            public static final Pattern constellationRegex = Pattern.compile("(?<!\\w)c(\\d+)");
            public static final Pattern skillLevelRegex = Pattern.compile("sl(\\d+)");
            public static final Pattern stateRegex = Pattern.compile("state(\\d+)");
            public static final Pattern blockRegex = Pattern.compile("blk(\\d+)");
            public static final Pattern groupRegex = Pattern.compile("grp(\\d+)");
            public static final Pattern configRegex = Pattern.compile("cfg(\\d+)");
            public static final Pattern hpRegex = Pattern.compile("(?<!\\w)hp(\\d+)");
            public static final Pattern maxHPRegex = Pattern.compile("maxhp(\\d+)");
            public static final Pattern atkRegex = Pattern.compile("atk(\\d+)");
            public static final Pattern defRegex = Pattern.compile("def(\\d+)");
            public static final Pattern aiRegex = Pattern.compile("ai(\\d+)");
            public static final Pattern sceneRegex = Pattern.compile("scene(\\d+)");
            public static final Pattern suiteRegex = Pattern.compile("suite(\\d+)");

            public static int matchIntOrNeg(Pattern pattern, String arg) {
                Matcher match = pattern.matcher(arg);
                if (match.find()) {
                    return Integer.parseInt(
                            match.group(
                                    1)); // This should be exception-safe as only \d+ can be passed to it (i.e. non-empty
                    // string of pure digits)
                }
                return -1;
            }

            public static <T> List<String> parseIntParameters(
                    List<String> args, @Nonnull T params, Map<Pattern, BiConsumer<T, Integer>> map) {
                args.removeIf(
                        arg -> {
                            var argL = arg.toLowerCase();
                            boolean deleteArg = false;
                            for (var entry : map.entrySet()) {
                                int argNum = matchIntOrNeg(entry.getKey(), argL);
                                if (argNum != -1) {
                                    entry.getValue().accept(params, argNum);
                                    deleteArg = true;
                                }
                            }
                            return deleteArg;
                        });
                return args;
            }

            public static float parseRelative(String input, Float current) {
                if (input.contains("~")) { // Relative
                    if (!input.equals("~")) { // Relative with offset
                        current += Float.parseFloat(input.replace("~", ""));
                    } // Else no offset, no modification
                } else { // Absolute
                    current = Float.parseFloat(input);
                }
                return current;
            }

            public static Position parsePosition(
                    String inputX, String inputY, String inputZ, Position curPos, Position curRot) {
                Position offset = new Position();
                Position target = new Position(curPos);
                if (inputX.contains("~")) { // Relative
                    if (!inputX.equals("~")) { // Relative with offset
                        target.addX(Float.parseFloat(inputX.replace("~", "")));
                    }
                } else if (inputX.contains("^")) {
                    if (!inputX.equals("^")) {
                        offset.setX(Float.parseFloat(inputX.replace("^", "")));
                    }
                } else { // Absolute
                    target.setX(Float.parseFloat(inputX));
                }

                if (inputY.contains("~")) { // Relative
                    if (!inputY.equals("~")) { // Relative with offset
                        target.addY(Float.parseFloat(inputY.replace("~", "")));
                    }
                } else if (inputY.contains("^")) {
                    if (!inputY.equals("^")) {
                        offset.setY(Float.parseFloat(inputY.replace("^", "")));
                    }
                } else { // Absolute
                    target.setY(Float.parseFloat(inputY));
                }

                if (inputZ.contains("~")) { // Relative
                    if (!inputZ.equals("~")) { // Relative with offset
                        target.addZ(Float.parseFloat(inputZ.replace("~", "")));
                    }
                } else if (inputZ.contains("^")) {
                    if (!inputZ.equals("^")) {
                        offset.setZ(Float.parseFloat(inputZ.replace("^", "")));
                    }
                } else { // Absolute
                    target.setZ(Float.parseFloat(inputZ));
                }

                if (!offset.equal3d(Position.ZERO)) {
                    return calculateOffset(target, curRot, offset);
                } else {
                    return target;
                }
            }

            public static Position calculateOffset(Position pos, Position rot, Position offset) {
                // Degrees to radians
                float angleZ = (float) Math.toRadians(rot.getY());
                float angleX = (float) Math.toRadians(rot.getY() + 90);

                // Calculate offset based on current position and rotation
                return new Position(
                        pos.getX()
                                + offset.getZ() * (float) Math.sin(angleZ)
                                + offset.getX() * (float) Math.sin(angleX),
                        pos.getY() + offset.getY(),
                        pos.getZ()
                                + offset.getZ() * (float) Math.cos(angleZ)
                                + offset.getX() * (float) Math.cos(angleX));
            }
        }

        """)

print(test.replace("\n", """
"""))

# listModels()


{'id': 'chatcmpl-99LLr6rMFOqZlFJV0Ij18FgCcnt7M', 'object': 'chat.completion', 'created': 1712014055, 'model': 'gpt-4-0125-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '```java\npackage emu.grasscutter.command.test;\n\nimport emu.grasscutter.command.CommandHelpers;\nimport emu.grasscutter.game.world.Position;\nimport org.junit.jupiter.api.Test;\nimport java.util.*;\nimport java.util.regex.*;\nimport java.util.function.BiConsumer;\nimport static org.junit.jupiter.api.Assertions.*;\n\nclass CommandHelpersTest {\n\n    @Test\n    void testMatchIntOrNeg() {\n        assertEquals(10, CommandHelpers.matchIntOrNeg(CommandHelpers.lvlRegex, "lvl10"));\n        assertEquals(-1, CommandHelpers.matchIntOrNeg(CommandHelpers.lvlRegex, "lvl"));\n        assertEquals(5, CommandHelpers.matchIntOrNeg(CommandHelpers.refineRegex, "r5"));\n        assertEquals(-1, CommandHelpers.matchIntOrNeg(CommandHelpers.lvlRegex, "abc"));\n    }\n\n    @Test\n    void testParseIntParame